# Electricity price forecasting


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import datetime

In [5]:
# import epftoolbox as epf
from epftoolbox.data import read_data

In [155]:
# Load the data
df_train, df_test = read_data(
    path='data_epf', dataset='FR', begin_test_date='01-01-2016',
    end_test_date='01-02-2016')

Test datasets: 2016-01-01 00:00:00 - 2016-02-01 23:00:00


In [156]:
df_test.head()


,Price,Exogenous 1,Exogenous 2
Date,,,
2016-01-01 00:00:00,23.86,58088.0,59108.0
2016-01-01 01:00:00,22.39,56761.0,54623.0
2016-01-01 02:00:00,20.59,55911.0,54673.0
2016-01-01 03:00:00,16.81,51949.0,50913.0
2016-01-01 04:00:00,17.41,51773.0,49368.0


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26280 entries, 2013-01-01 00:00:00 to 2015-12-31 23:00:00
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Price        26280 non-null  float64
 1   Exogenous 1  26280 non-null  float64
 2   Exogenous 2  26280 non-null  float64
dtypes: float64(3)
memory usage: 821.2 KB


In [37]:
df_train.index

DatetimeIndex(['2013-01-01 00:00:00', '2013-01-01 01:00:00',
               '2013-01-01 02:00:00', '2013-01-01 03:00:00',
               '2013-01-01 04:00:00', '2013-01-01 05:00:00',
               '2013-01-01 06:00:00', '2013-01-01 07:00:00',
               '2013-01-01 08:00:00', '2013-01-01 09:00:00',
               ...
               '2015-12-31 14:00:00', '2015-12-31 15:00:00',
               '2015-12-31 16:00:00', '2015-12-31 17:00:00',
               '2015-12-31 18:00:00', '2015-12-31 19:00:00',
               '2015-12-31 20:00:00', '2015-12-31 21:00:00',
               '2015-12-31 22:00:00', '2015-12-31 23:00:00'],
              dtype='datetime64[ns]', name='Date', length=26280, freq=None)

In [114]:
def preprocess(df, cols_countries, pivot=False):

    # get the FR and DE string from cols_FR_DE
    countries = [col.split('_')[0] for col in cols_countries]
    countries = list(set(countries))
    countries.remove('DAH') 
    # Convert the date to datetime   
    df['date'] = pd.to_datetime(df['date'])
    # Extracting Year, Month, Day, Day of Week, and Hour
    df['Year'] = df['date'].dt.year
    df['Month'] = df['date'].dt.month
    df['Day'] = df['date'].dt.day
    df['DayOfWeek'] = df['date'].dt.day_name()
    df['Hour'] = df['date'].dt.hour
    df['Hour'] = df['Hour'].apply(lambda x: str(x + 1).zfill(2) if x < 23 else '00')
    df.insert(0, 'Hour', df.pop('Hour'))
    df.insert(0, 'DayOfWeek', df.pop('DayOfWeek'))
    df.insert(0, 'Day', df.pop('Day'))
    df.insert(0, 'Month', df.pop('Month'))
    df.insert(0, 'Year', df.pop('Year'))

    # Dataset for FR and DE 
    
    df_FR_DE = df[['Year', 'Month', 'Day', 'DayOfWeek', 'Hour', 'date'] + cols_countries]

    # We drop the NaN values 
    df_FR_DE.dropna(how='all', inplace=True)
    
    if pivot is True:

        # Colonne Y_M_D pour pouvoir grouper les données sur un même jour
        df_FR_DE['Y_M_D'] = df_FR_DE['Year'].astype('str') + "_" + df_FR_DE['Month'].astype('str') + "_" + df_FR_DE['Day'].astype('str')
        df_FR_DE['Y_M_D_H'] = df_FR_DE['Year'].astype('str') + "_" + df_FR_DE['Month'].astype('str') + "_" + df_FR_DE['Day'].astype('str')+ "_" + df_FR_DE['Hour'].astype('str')
        # On supprime les duplicates
        df_FR_DE.drop_duplicates(subset=["Y_M_D_H"], inplace=True)

        # Ajout des colonnes FR_Spot_J1 et DE_Spot_J1
        df_FR_DE_J1 = df_FR_DE.copy()[[countries[0]+"_Spot", countries[1]+"_Spot", "date"]]

        df_FR_DE_J1["date+1"] = df_FR_DE_J1["date"] + datetime.timedelta(days=1)
        df_FR_DE_J1.rename(columns={countries[0]+"_Spot":countries[0]+"_Spot_J1", countries[1]+"_Spot":countries[1]+"_Spot_J1"}, inplace=True)
        df_FR_DE = pd.merge(df_FR_DE, df_FR_DE_J1.drop(columns=['date']), left_on="date", right_on="date+1")

        # Table pivot : 1 ligne = 1 jour avec 24*10 features pour chaque heure de la journée
        df_FR_DE_pivot = pd.pivot_table(df_FR_DE, columns=["Hour"], values=cols_countries+[countries[0]+'_Spot_J1', countries[1]+'_Spot_J1'], index=['Y_M_D'], aggfunc='sum')
        df_FR_DE_pivot.columns = df_FR_DE_pivot.columns.to_flat_index()
        df_FR_DE_pivot.reset_index(inplace=True)
        list_col = df_FR_DE_pivot.columns.values.tolist()
        new_cols = []
        for col in list_col:
            col = str(col)
            col = col.replace('(', '').replace(')', '').replace(",", "").replace(" ", "_").replace("'", "")
            new_cols.append(col)
            
        df_FR_DE_pivot.columns = new_cols

        df_FR_DE_pivot.dropna(inplace=True)
        df_FR_DE = df_FR_DE_pivot
        

    return df_FR_DE

Here, we load data from Olivier

In [138]:
df = pd.read_csv('data\Data_Europe_DAH_2016_2023.csv', index_col=0, parse_dates=True)
# Choose the countries and the columns
cols_FR_DE = [
    'FR_Spot', 'FR_Load', 'DE_Spot', 'DE_Load',
    'FR_DAH_generation', 'DE_DAH_generation',
    # 'FR_wind onshore_DAH', 'DE_wind onshore_DAH', 
    'DAH_ImportCapacity_FR_DE', 'DAH_ImportCapacity_DE_FR'
             ]
# Pivot = True pour avoir une ligne par jour
df_FR_DE = preprocess(df, cols_FR_DE, pivot=False)
df_FR_DE.set_index('date', inplace=True, drop=True)

C:\Users\matis\AppData\Local\Temp\ipykernel_26620\60855226.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('data\Data_Europe_DAH_2016_2023.csv', index_col=0, parse_dates=True)
C:\Users\matis\AppData\Local\Temp\ipykernel_26620\1255989459.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_FR_DE.dropna(how='all', inplace=True)


In [139]:
min_date = df_FR_DE.index.min()
print(min_date)
max_date = df_FR_DE.index.max()
print(max_date)

2016-01-01 00:00:00
2023-12-31 23:00:00


Keep only numerical values

In [140]:
# numerical columns
cols = df_FR_DE.columns
num_cols = list(df_FR_DE._get_numeric_data().columns)
num_cols.remove('Year')
num_cols.remove('Month')
num_cols.remove('Day')
cat_cols = list(set(cols) - set(num_cols))
df_FR_DE_num = df_FR_DE[num_cols]


In [141]:
df_FR_DE_num.tail()

,FR_Spot,FR_Load,DE_Spot,DE_Load,FR_DAH_generation,DE_DAH_generation,DAH_ImportCapacity_FR_DE,DAH_ImportCapacity_DE_FR
date,,,,,,,,
2023-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Format dataframe with columns (Price, Exogenous 1, etc.)

In [142]:
# columns except DE_Spot
cols_FR = [col for col in df_FR_DE_num.columns if col != 'DE_Spot']
df_FR = df_FR_DE_num[cols_FR]
# list of columns (Price, Exogenous1, Exogenous2, ...)
new_cols= ['Price']+[f'Exogenous {i}' for i in range(1, len(cols_FR))]
df_FR.columns = new_cols

In [143]:
df_FR.head()


,Price,Exogenous 1,Exogenous 2,Exogenous 3,Exogenous 4,Exogenous 5,Exogenous 6
date,,,,,,,
2016-01-01 00:00:00,22.39,56550.0,177664.36,62205.0,NaN,NaN,NaN
2016-01-01 01:00:00,20.59,56150.0,171405.81,60615.0,NaN,NaN,NaN
2016-01-01 02:00:00,16.81,52600.0,169234.01,56583.5,NaN,NaN,NaN
2016-01-01 03:00:00,17.41,49750.0,169488.24,56541.0,NaN,NaN,NaN
2016-01-01 04:00:00,17.02,48850.0,172496.31,56919.0,NaN,NaN,NaN


Fill missing values with median per month 

In [144]:
# Fill na with median of column on 1 month 
# Resample DataFrame to a monthly frequency
df_FR_monthly = df_FR.resample('M').median()
# Fill missing values with the median of each column calculated over a rolling window of 1 month
df_FR.fillna(df_FR_monthly.median(), inplace=True)

C:\Users\matis\AppData\Local\Temp\ipykernel_26620\2383202340.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_FR.fillna(df_FR_monthly.median(), inplace=True)


In [145]:
df_FR.tail()

,Price,Exogenous 1,Exogenous 2,Exogenous 3,Exogenous 4,Exogenous 5,Exogenous 6
date,,,,,,,
2023-12-31 19:00:00,50.695,49537.5,232007.175,56211.125,53744.2975,971.525,0.0
2023-12-31 20:00:00,50.695,49537.5,232007.175,56211.125,53744.2975,971.525,0.0
2023-12-31 21:00:00,50.695,49537.5,232007.175,56211.125,53744.2975,971.525,0.0
2023-12-31 22:00:00,50.695,49537.5,232007.175,56211.125,53744.2975,971.525,0.0
2023-12-31 23:00:00,50.695,49537.5,232007.175,56211.125,53744.2975,971.525,0.0


Export to csv file

In [146]:
df_FR.to_csv('datasets\FR_readytodnn.csv')

Train and test split
- 4 years to train
- minimum 1 year for test
- /!\ 2021, 2022 prediction, covid 

In [66]:
begin_date_test = '2023-09-27'
end_date_test = '2023-11-27'
df_test = df_FR_DE_num.loc[begin_date_test:end_date_test]
df_train = df_FR_DE_num.loc[:begin_date_test]

Scaling data with EPFtoolbox :

    'Norm' for normalizing the data to the interval [0, 1].
    'Norm1' for normalizing the data to the interval [-1, 1].
    'Std' for standarizing the data to follow a normal distribution.
    'Median' for normalizing the data based on the median as defined in as defined in here.
    'Invariant' for scaling the data based on the asinh transformation (a variance stabilizing transformations) as defined in here.


In [67]:
from epftoolbox.data import DataScaler
Xtrain = df_train.values
Xtest = df_test.values
scaler = DataScaler('Norm')
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

# Xtrain_inverse = scaler.inverse_transform(Xtrain_scaled)
# Xtest_inverse = scaler.inverse_transform(Xtest_scaled)


In [71]:
df_train.tail()

,Year,Month,Day,FR_Spot,FR_Load,DE_Spot,DE_Load,FR_DAH_generation,DE_DAH_generation,DAH_ImportCapacity_FR_DE,DAH_ImportCapacity_DE_FR
date,,,,,,,,,,,
2023-09-27 19:00:00,2023,9,27,106.28,45200.0,107.82,207487.66,48853.0,44660.04,0.0,1812.5
2023-09-27 20:00:00,2023,9,27,96.38,43500.0,96.38,191913.72,48365.5,43001.17,0.0,1413.6
2023-09-27 21:00:00,2023,9,27,75.98,44450.0,89.09,178463.14,47612.0,41629.06,0.0,2240.6
2023-09-27 22:00:00,2023,9,27,79.61,41450.0,88.36,172624.22,44330.5,39404.32,0.0,2855.3
2023-09-27 23:00:00,2023,9,27,65.54,38550.0,84.05,166243.75,43053.5,38636.27,0.0,2997.8


## The input features that are considered in Benchmark model

#### EPFtoolbox datasets : 
https://arxiv.org/pdf/2008.08004.pdf

Independently of the model, the available input features to forecast the 24 day-ahead prices of day d,
i.e. pd = [pd,1, . . . , pd,24], are the same:

    - Historical day-ahead prices of the previous three days and one week ago, i.e. pd−1, pd−2, pd−3, pd−7.

    - The day-ahead forecasts of the two variables of interest (see Section 3 for details) for day d available on day d − 1, i.e. x1d = [x1d,1, . . . , x1d,24] and x2d = [x2d,1, . . . , x2d,24] ; note that the variables of interest are different for each market.

    - Historical day-ahead forecasts of the variables of interest the previous day and one week ago, i.e. x1d−1,x1d−7, x2d−1, x2d−7.
    
    - A dummy variable zd that represents the day of the
    week. In the case of the linear model, following
    the standard practice in the literature [55, 58, 77],

Made in function **_build_and_split_XYs**

#### SOTA datasets + enriched data with circular encoding information
https://hal.science/hal-03621974v1/document

Electricity price datasets are a multivariate time series made of daily
data. Those datasets can be reconfigured into a (𝑋, 𝑌 ) couple suitable
to learn machine learning models. The predictive data is represented
by a two dimensional matrix 𝑋 ∈ R𝑛𝑑×𝑛𝑐 whose rows represent days
and columns are 𝑛𝑐 predictive time-dependent values. The values to
be predicted correspond to another matrix 𝑌 ∈ R𝑛𝑑×𝑛𝑜 , whose rows
also stand for the days and columns are the 𝑛𝑜 day-ahead prices to be
predicted: 𝑌𝑑 =(𝑌1𝑑+1, …, 𝑌 𝑛0𝑑+1).

To model the time series aspect of the features, 𝑋 includes the prices of the current day, those of the day
before, two days before and the previous week (1, 2, 3 and 7 days lag).
Exogenous features are included for the day, the day before and the
previous week. In addition to these 240 characteristics, the day of the
week is also encoded as an integer and added to the matrix 𝑋. Indeed,
electricity prices are non-stationary time series and exhibit seasonal
trends captured by this additional feature. All features (prices and
exogenous) are provided with hourly granularity. Thus, the predictive
matrix 𝑋 is as follows:
𝑋𝑑 =(𝑌𝑑−1, 𝑌𝑑−2, 𝑌𝑑−3, 𝑌𝑑−7, 𝐸1𝑑, 𝐸1𝑑−1, 𝐸1𝑑−7,𝐸2𝑑, 𝐸2𝑑−1, 𝐸2𝑑−7,DayOfWeek) with 𝑛𝑐=241.

In order to forecast 24-hour prices for the next day, the datasets are
reshaped so that for one day 𝑑, 𝑌𝑑
contains all 24 prices for the next
day: 𝑌𝑑 =(𝑌1𝑑+1, …, 𝑌 24𝑑+1).



### Hyperparameter optimization
For the DNN, as in the original study [59], the input
features are optimized together with the hyperparameters using the tree Parzen estimator [108] (see Section
4.3 for details).

As in the original DNN paper [59], the hyperparameters
and input features are optimized together using the treestructured Parzen estimator [108], a Bayesian optimization
algorithm based on sequential model-based optimization.
To do so, the features are modeled as hyperparameters,
with each hyperparameter representing a binary variable
that selects whether or not a specific feature is included in
the model (as explained in [43]). In more detail, to select
which of the 241 available input features are relevant, the
method employs 11 decision variables, i.e. 11 hyperparameters:

    - Four binary hyperparameters (1-4) that indicate
    whether or not to include the historical day ahead
    prices pd−1, pd−2, pd−3, pd−7. The selection is done
    per day12, e.g. the algorithm either selects all the
    prices pd−j of j days ago or it cannot select any price
    from day d − j, hence the four hyperparameters.

    - Two binary hyperparameters (5-6) that indicate
    whether or not to include each of the day-ahead forecasts x1d and x2d. As with the past prices, this is done for the whole day, i.e. a hyperparameter either selects all the elements in xjd or none.

    - Four binary hyperparameters (7-10) that indicate
    whether or not to include the historical day-ahead
    forecasts x1d−1, x2d−1, x1d−7, and x2d−7. This selection is also done per day.

    - One binary hyperparameter (11) that indicates
    whether or not to include the variable zd representing
    the day of the week.
In short, 10 binary hyperparameters indicating whether or
not to include 24 inputs each and another binary hyperparameter indicating whether or not to include a dummy
variable.

Besides selecting the features, the algorithm also optimizes eight additional hyperparameters:
1) the number of neurons per layer
2) the activation function
3) the dropout rate
4) the learning rate
5) whether or not to
use batch normalization
6) the type of data preprocessing technique
7) the initialization of the DNN weights
8) the coefficient for L1 regularization that is applied to each
layer’s kernel.

Unlike the weights of the DNN that are recalibrated on a
daily basis, the hyperparameter and features are optimized
only once using the four years of data prior to the testing
period. 

It is important to note that the algorithm runs
for a number T of iterations, where at every iteration the
algorithm infers a potential optimal subset of hyperparameters/features and evaluates this subset in the validation dataset.

For the proposed open-access benchmark models, T is selected as 1500 iterations to obtain a trade-off
between accuracy and computational requirements.


In [148]:
X = pd.read_csv("datasets/FR_readytodnn.csv", index_col=0, parse_dates=True)
min_date = X.index.min()
print(min_date)
max_date = X.index.max()
print(max_date)

2016-01-01 00:00:00
2023-12-31 23:00:00


In [149]:
from epftoolbox.models import hyperparameter_optimizer

# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = 'FR_readytodnn'

# Number of years (a year is 364 days) in the test dataset.
years_test = 1

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "2022-12-31"
end_test_date = "2023-12-31"

# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = 4

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 1

# Number of iterations for hyperparameter optimization
# It can be empirically observed that the performance of the models barely improves after 1000 iterations. 
# Moreover, performing 1500 iteration takes approximately just one day on a regular quadcore
# laptop like the i7-6920HQ, a computation cost very acceptable when
# the algorithm has to run only once
max_evals = 1500

path_datasets_folder = "./datasets/"
path_hyperparameters_folder = "./experimental_files/"


# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 1 year of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder, 
                         path_hyperparameters_folder=path_hyperparameters_folder, 
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset, 
                         years_test=years_test, calibration_window=calibration_window, 
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

c:\Users\matis\anaconda3\envs\ds\lib\site-packages\epftoolbox\data\_datasets.py:146: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  begin_test_date = pd.to_datetime(begin_test_date, dayfirst=True)
c:\Users\matis\anaconda3\envs\ds\lib\site-packages\epftoolbox\data\_datasets.py:147: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  end_test_date = pd.to_datetime(end_test_date, dayfirst=True)


Test datasets: 2022-12-31 00:00:00 - 2023-12-31 23:00:00





Tested 1/1500 iterations.
Best MAE - Validation Dataset
  MAE: 38.6 | sMAPE: 33.67 %

Best MAE - Test Dataset
  MAE: 24.4 | sMAPE: 34.54 %




Tested 2/1500 iterations.
Best MAE - Validation Dataset
  MAE: 38.6 | sMAPE: 33.67 %

Best MAE - Test Dataset
  MAE: 24.4 | sMAPE: 34.54 %




Tested 3/1500 iterations.
Best MAE - Validation Dataset
  MAE: 38.6 | sMAPE: 33.67 %

Best MAE - Test Dataset
  MAE: 24.4 | sMAPE: 34.54 %


c:\Users\matis\anaconda3\envs\ds\lib\site-packages\epftoolbox\data\_wrangling.py:49: RuntimeWarning: divide by zero encountered in divide
  transformed_data[:, i] = (data[:, i] - self.median[i]) / self.mad[i]
c:\Users\matis\anaconda3\envs\ds\lib\site-packages\epftoolbox\data\_wrangling.py:49: RuntimeWarning: invalid value encountered in divide
  transformed_data[:, i] = (data[:, i] - self.median[i]) / self.mad[i]




Tested 4/1500 iterations.
Best MAE - Validation Dataset
  MAE: 38.6 | sMAPE: 33.67 %

Best MAE - Test Dataset
  MAE: 24.4 | sMAPE: 34.54 %


c:\Users\matis\anaconda3\envs\ds\lib\site-packages\epftoolbox\data\_wrangling.py:49: RuntimeWarning: divide by zero encountered in divide
  transformed_data[:, i] = (data[:, i] - self.median[i]) / self.mad[i]
c:\Users\matis\anaconda3\envs\ds\lib\site-packages\epftoolbox\data\_wrangling.py:49: RuntimeWarning: invalid value encountered in divide
  transformed_data[:, i] = (data[:, i] - self.median[i]) / self.mad[i]


KeyboardInterrupt: 

## Checklist to ensure adequate EPF research :

As a final contribution, and with the goal of facilitating
the work of reviewers of future EPF publications, we provide a short checklist to evaluate whether any new research in EPF satisfies the requirements to be reproducible and to lead to meaningful conclusions:

    • The test dataset comprises at least a year of data.
    • Any new model is tested against state-of-the-art open-access models, e.g. the ones provided here.
    • The computational cost of new methods is evaluated
    and compared against the computational cost of existing methods.
    • The employed datasets are open-access.
    • The study is based on multiple markets.
    • rMAE is employed as one of the accuracy metrics to
    evaluate forecasting accuracy.
    • Statistical testing is used to assess whether differences
    in performance are significant.
    • Forecasting models are recalibrated on a daily basis
    and not simply estimated once and evaluated in the
    full out-of-sample dataset.
    • Hyperparameters are estimated using a validation
    dataset that is different from the test dataset.
    • The split and dates of the dataset are explicitly stated.
    • All the inputs of the model are explicitly defined
    - The test dataset is selected as the last section of the
    full dataset and does not contain any overlapping data
    with the training or validation datasets.
    • State-of-the-art and free toolboxes are used for modeling the benchmark models.